# Store Item Demand Forecasting Challenge
- predict 3 months of item sales at different stores
- https://www.kaggle.com/competitions/demand-forecasting-kernels-only/overview

**Continuing from previous notebook, we will implement individual SARIMAX(3,1,1)(1,1,1,7) to all item-store sets, then combine the results for submission.**

**For each item-store set, will provide as exogenous data: the seasonal data from seasonal_decompose of period=365 and period=7 days, and boolean masks of special days (holidays and 7 days leading to holidays)**

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import re
import warnings 
warnings.filterwarnings('ignore')

test  = pd.read_csv('./data/test.csv', index_col=[0])
train = pd.read_csv('./data/train.csv', )

train['date'] = pd.to_datetime(train['date'], format="%Y-%m-%d")
test['date']  = pd.to_datetime(test['date'], format="%Y-%m-%d")
cutoff=len(train)

combined = pd.concat([train,test])
fulllength = len(combined)


###? PREP EXOG FEATURES: 

#? mark US holidays, also mark 7 days before holidays as 'preholidays' as people may buy items before the holidays.
holidays = pd.read_csv('./data/US Holiday Dates (2004-2021).csv')
holidays.set_index('Date', inplace=True)

def addholidays_fulldf(df):
    df['holiday']    =0
    df['preholiday'] =0
    for d in holidays.index:
        d = pd.to_datetime(d)
        if d in df['date'].values:
            df.loc[df['date']==pd.to_datetime(d), 'holiday']=1        
            
        if pd.to_datetime(d)-pd.Timedelta(7, "d") in df['date'].values:
            df.loc[df['date']==(pd.to_datetime(d)-pd.Timedelta(1, "d")), 'preholiday']=1
            df.loc[df['date']==(pd.to_datetime(d)-pd.Timedelta(2, "d")), 'preholiday']=1
            df.loc[df['date']==(pd.to_datetime(d)-pd.Timedelta(3, "d")), 'preholiday']=1
            df.loc[df['date']==(pd.to_datetime(d)-pd.Timedelta(4, "d")), 'preholiday']=1
            df.loc[df['date']==(pd.to_datetime(d)-pd.Timedelta(5, "d")), 'preholiday']=1
            df.loc[df['date']==(pd.to_datetime(d)-pd.Timedelta(6, "d")), 'preholiday']=1
            df.loc[df['date']==(pd.to_datetime(d)-pd.Timedelta(7, "d")), 'preholiday']=1
    return df

combined = addholidays_fulldf(combined)
train = combined.iloc[:cutoff]
test = combined.iloc[cutoff:]


###? PART OUT INDIVIDUAL STORE AND ITEMS INTO INDIVIDUAL TIME SERIES. E.G. i50s10 for item 50, store 10
s_train = []
for item in train['item'].unique():
    for store in train['store'].unique():
        sname = 'train' + str(item).rjust(2, '0') + 's' + str(store).rjust(2, '0')
        tmp = train[(train['item']==item)&(train['store']==store)].copy(deep=True)
        tmp.set_index('date', inplace=True)
        globals()[sname] = pd.DataFrame( {'sales':tmp['sales'],'holiday':tmp['holiday'],'preholiday':tmp['preholiday']} )
        s_train.append(sname)

s_test = []
for item in test['item'].unique():
    for store in test['store'].unique():
        sname = 'test' + str(item).rjust(2, '0') + 's' + str(store).rjust(2, '0')
        tmp = test[(test['item']==item)&(test['store']==store)].copy(deep=True)
        tmp.set_index('date', inplace=True)
        globals()[sname] = pd.DataFrame( {'sales':tmp['sales'],'holiday':tmp['holiday'],'preholiday':tmp['preholiday']} )
        s_test.append(sname)



#? provide 365 day seasonal, 7 day seasonal,
s_train_sd_yrly = []
s_train_sd_wkly = []
for item in s_train:  
    # trn = eval(item)[:1461]
    # tes = eval(item)[1461:]

    sd_yrly = sm.tsa.seasonal_decompose(eval(item)['sales'], model='additive', period=365) # sd.trend # sd.resid # sd.seasonal
    sd_wkly = sm.tsa.seasonal_decompose(eval(item)['sales'], model='additive', period=7)
    s_train_sd_yrly.append(sd_yrly.seasonal)
    s_train_sd_wkly.append(sd_wkly.seasonal)

In [2]:
startdate = test['date'].min()
enddate = test['date'].max()

full_predictions = pd.DataFrame()

for i, item in enumerate(s_train):

    product = item.split('train')[1].split('s')[0]
    store   = item.split('train')[1].split('s')[1]

    globals()[item]['sd_yrly'] = s_train_sd_yrly[i]
    globals()[item]['sd_wkly'] = s_train_sd_wkly[i]
    # display(globals()[item])

    endog = globals()[item]['sales']
    exog  = globals()[item][['holiday','preholiday','sd_yrly','sd_wkly']]

    exog2013 = exog.loc['2013'].copy(deep=True)
    exog2014 = exog.loc['2014'].copy(deep=True)
    exog2015 = exog.loc['2015'].copy(deep=True)
    exog2016 = exog.loc['2016'].copy(deep=True)

    exog2014.index = exog2014.index - pd.Timedelta(365, 'days')
    exog2015.index = exog2015.index - pd.Timedelta(365*2, 'days')
    exog2016.index = exog2016.index - pd.Timedelta(365*3, 'days')

    exog2013.reset_index(inplace=True)
    exog2014.reset_index(inplace=True)
    exog2015.reset_index(inplace=True)
    exog2016.reset_index(inplace=True)

    exogagg = pd.concat([exog2013,exog2014,exog2015,exog2016])
    exogpred = exogagg.groupby('date').mean()

    exog_test = test[(test['item']==1)&(test['store']==1)][['holiday','preholiday']]
    exog_test['sd_yrly'] = exogpred['sd_yrly'][:len(exog_test)].values
    exog_test['sd_wkly'] = exogpred['sd_wkly'][:len(exog_test)].values

    # model = sm.tsa.statespace.SARIMAX(endog.asfreq(freq='1d'), order=(3,1,1), seasonal_order=(2,1,0,7), trend=None, exog=exog).fit(method='cg', max_iter=100)
    model = sm.tsa.statespace.SARIMAX(endog, order=(3,1,1), seasonal_order=(1,1,1,7), trend=None, exog=exog).fit()
    predictions = model.predict(start=startdate, end=enddate, exog=exog_test)
    
    pred_df = pd.DataFrame({'sales':predictions})
    pred_df['item']=int(product)
    pred_df['store']=int(store)
    
    full_predictions = pd.concat([full_predictions, pred_df])


###? UPDATE PREDICTIONS INTO TEST DF.
for row in full_predictions.iterrows():
    # print(row[0], row[1]['sales'], row[1]['item'], row[1]['store'])
    test.loc[\
        (test['store']==row[1]['store'])&\
        (test['item']==row[1]['item'])&\
        (test['date']==row[0]) , 'sales'] = row[1]['sales']


test['sales'].to_csv('storesales2.csv')


**Kaggle Scores**
- **Public Score: 17.39495 (345/459), Private score: 14.92581 (316/459)**
- (This leaderboard is calculated with approximately 34% of the test data. The final results will be based on the other 66%.)
- aggregated across complete test set, that's 15.76531 SMAPE score.


In [43]:
17.39495*0.34 + 14.92581*0.66

15.765317600000001